In [1]:
from mbientlab.metawear import MetaWear, libmetawear
from mbientlab.metawear.cbindings import *
from ctypes import c_void_p, POINTER, CFUNCTYPE
import time

In [25]:
# MAC address of the MetaWear device
device_address = 'D4:06:BD:85:8F:23'

# Create and connect the device
device = MetaWear(device_address)
device.connect()

In [26]:
data_list = []

# Callback function to handle incoming data
def data_handler(context, data):
    # Convert the data pointer to a usable format, assuming it's CartesianFloat for demonstration
    if data.contents.type_id == DataTypeId.CARTESIAN_FLOAT:
        value = cast(data.contents.value, POINTER(CartesianFloat)).contents
        print("X: %.2f Y: %.2f Z: %.2f" % (value.x, value.y, value.z))
        data_list.append([value.x, value.y, value.z])

In [27]:

# Define the callback type
FnVoid_VoidP_DataP = CFUNCTYPE(None, c_void_p, POINTER(Data))

# Setup callback
callback = FnVoid_VoidP_DataP(data_handler)

# Configure accelerometer
libmetawear.mbl_mw_acc_set_odr(device.board, 100.0)  # Set output data rate to 100 Hz
libmetawear.mbl_mw_acc_set_range(device.board, 16.0)  # Set range to +/-16g
libmetawear.mbl_mw_acc_write_acceleration_config(device.board)

# Get the accelerometer data signal
acc_signal = libmetawear.mbl_mw_acc_get_acceleration_data_signal(device.board)



libmetawear.mbl_mw_gyro_bmi160_write_config(device.board)
gyro_signal = libmetawear.mbl_mw_gyro_bmi160_get_rotation_data_signal(device.board)



# Subscribe to the accelerometer data signal
libmetawear.mbl_mw_datasignal_subscribe(acc_signal, None, callback)
libmetawear.mbl_mw_datasignal_subscribe(gyro_signal, None, callback)


# Enable accelerometer
libmetawear.mbl_mw_acc_enable_acceleration_sampling(device.board)
libmetawear.mbl_mw_acc_start(device.board)
libmetawear.mbl_mw_gyro_bmi160_enable_rotation_sampling(device.board)
libmetawear.mbl_mw_gyro_bmi160_start(device.board)

# Stream data for a duration (e.g., 30 seconds)
time.sleep(5)

# Stop and clean up
libmetawear.mbl_mw_acc_stop(device.board)
libmetawear.mbl_mw_acc_disable_acceleration_sampling(device.board)
libmetawear.mbl_mw_datasignal_unsubscribe(acc_signal)
libmetawear.mbl_mw_debug_disconnect(device.board)
print("Disconnected and cleaned up")

X: -0.06 Y: 0.49 Z: 0.06
X: -0.12 Y: 0.43 Z: 0.06
X: -0.18 Y: 0.49 Z: 0.00
X: -0.12 Y: 0.49 Z: -0.18
X: -0.18 Y: 0.43 Z: -0.06
X: -0.18 Y: 0.43 Z: 0.00
X: -0.12 Y: 0.37 Z: -0.12
X: -0.24 Y: 0.43 Z: -0.12
X: -0.12 Y: 0.37 Z: 0.00
X: -0.24 Y: 0.43 Z: 0.30
X: -0.06 Y: 0.43 Z: -0.06
X: -0.12 Y: 0.43 Z: 0.00
X: -0.24 Y: 0.49 Z: 0.43
X: -0.12 Y: 0.43 Z: 0.18
X: -0.18 Y: 0.43 Z: -0.18
X: -0.18 Y: 0.37 Z: 0.12
X: -0.12 Y: 0.43 Z: 0.12
X: -0.12 Y: 0.43 Z: -0.18
X: -0.06 Y: 0.55 Z: -0.24
X: -0.12 Y: 0.49 Z: 0.00
X: 0.00 Y: -1.05 Z: -0.01
X: -0.18 Y: 0.43 Z: 0.12
X: 0.00 Y: -1.05 Z: -0.01
X: -0.18 Y: 0.43 Z: 0.00
X: 0.00 Y: -1.05 Z: -0.01
X: -0.06 Y: 0.43 Z: -0.18
X: -0.00 Y: -1.05 Z: -0.01
X: -0.12 Y: 0.49 Z: 0.18
X: -0.00 Y: -1.05 Z: -0.01
X: 0.00 Y: 0.49 Z: 0.18
X: -0.01 Y: -1.05 Z: -0.01
X: -0.12 Y: 0.49 Z: 0.12
X: -0.01 Y: -1.05 Z: -0.01
X: 0.00 Y: 0.49 Z: 0.06
X: -0.01 Y: -1.05 Z: -0.01
X: -0.06 Y: 0.49 Z: 0.06
X: -0.01 Y: -1.05 Z: -0.01
X: -0.12 Y: 0.49 Z: -0.06
X: -0.00 Y: -1.05 Z: -0.01


In [22]:
device.on_disconnect = lambda status: print ("we are disconnected!")
device.disconnect()

we are disconnected!


In [11]:
print(data_list[0:11])

[]
